In [1]:
import boto3
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()  # Use the AWS region configured with the AWS CLI
# sess = sagemaker.Session(boto3.session.Session(region_name='eu-west-1'))

# This doesn't work on your local machine because it doesn't have an IAM role :)
# role = sagemaker.get_execution_role()

# This is the SageMaker role you're already using, it will work just fine
role = "arn:aws:iam::390844780979:role/service-role/AmazonSageMaker-ExecutionRole-20250403T173454"

2.183.0


In [2]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist

(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

Using TensorFlow backend.
OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-15
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 16 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #192: KMP_AFFINITY: 1 socket x 8 cores/socket x 2 threads/core (8 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 2 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 3 maps to socket 0 core 1 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 4 maps to socket 0 core 2 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 5 maps to socket 0 core 2 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 6 maps to socket 0 core 3 thread 0 
OMP: I

In [3]:
# Train on local data. S3 URIs would work too.
training_input_path   = 'file://data/training.npz'
validation_input_path = 'file://data/validation.npz'

# Store model locally. A S3 URI would work too.
output_path           = 'file:///tmp/model/'

In [8]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist_keras_tf.py',
                          role=role,
                          instance_count=1, 
                          instance_type='local',   # Train on the local CPU ('local_gpu' if it has a GPU)
                          framework_version='1.15', 
                          py_version='py3',
                          hyperparameters={'epochs': 4, "learning-rate": 0.01},
                          output_path=output_path
                         )

/mnt/d/AWS/fraud-detection/.env/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [9]:
# Train! This will pull (once) the SageMaker CPU/GPU container for TensorFlow to your local machine.
# Make sure that Docker is running and that docker-compose is installed

tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2025-04-15-10-38-20-892
INFO:sagemaker.local.local_session:Starting training job
INFO:sagemaker.local.image:Using the long-lived AWS credentials found in session
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-i0zf2:
    command: train
    container_name: qz4i0ko4pc-algo-1-i0zf2
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:1.15-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-i0zf2
    std

time="2025-04-15T11:38:29+01:00" level=warning msg="/tmp/tmpsse2jnk6/docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
time="2025-04-15T11:38:30+01:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmpsse2jnk6\".\nSet `external: true` to use an existing network"
 Container qz4i0ko4pc-algo-1-i0zf2  Creating
 Container qz4i0ko4pc-algo-1-i0zf2  Created
Attaching to qz4i0ko4pc-algo-1-i0zf2
qz4i0ko4pc-algo-1-i0zf2  | WARNING:tensorflow:From /usr/local/lib/python3.6/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
qz4i0ko4pc-algo-1-i0zf2  | 
qz4i0ko4pc-algo-1-i0zf2  | 2025-04-15 10:38:33,208 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
qz4i0ko4pc-algo-1-i0zf2  | 2025-04-15 10:38:33,216 sagemaker-training-toolkit INFO     No GPUs

qz4i0ko4pc-algo-1-i0zf2  | channels_last
qz4i0ko4pc-algo-1-i0zf2  | x_train shape: (60000, 28, 28, 1)
qz4i0ko4pc-algo-1-i0zf2  | 60000 train samples
qz4i0ko4pc-algo-1-i0zf2  | 10000 test samples
qz4i0ko4pc-algo-1-i0zf2  | Model: "sequential_1"
qz4i0ko4pc-algo-1-i0zf2  | _________________________________________________________________
qz4i0ko4pc-algo-1-i0zf2  | Layer (type)                 Output Shape              Param #   
qz4i0ko4pc-algo-1-i0zf2  | =================================================================
qz4i0ko4pc-algo-1-i0zf2  | conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
qz4i0ko4pc-algo-1-i0zf2  | _________________________________________________________________
qz4i0ko4pc-algo-1-i0zf2  | batch_normalization_1 (Batch (None, 28, 28, 64)        256       
qz4i0ko4pc-algo-1-i0zf2  | _________________________________________________________________
qz4i0ko4pc-algo-1-i0zf2  | activation_1 (Activation)    (None, 28, 28, 64)        0         
qz4i0ko4pc-a

INFO:root:creating /tmp/tmpsse2jnk6/artifacts/output/data
INFO:root:copying /tmp/tmpsse2jnk6/algo-1-i0zf2/output/success -> /tmp/tmpsse2jnk6/artifacts/output
INFO:root:creating /tmp/tmpsse2jnk6/artifacts/model/model
INFO:root:creating /tmp/tmpsse2jnk6/artifacts/model/model/1
INFO:root:creating /tmp/tmpsse2jnk6/artifacts/model/model/1/variables
INFO:root:copying /tmp/tmpsse2jnk6/model/model/1/variables/variables.index -> /tmp/tmpsse2jnk6/artifacts/model/model/1/variables
INFO:root:copying /tmp/tmpsse2jnk6/model/model/1/variables/variables.data-00000-of-00001 -> /tmp/tmpsse2jnk6/artifacts/model/model/1/variables


qz4i0ko4pc-algo-1-i0zf2 exited with code 0
Aborting on container exit...
 Container qz4i0ko4pc-algo-1-i0zf2  Stopping
 Container qz4i0ko4pc-algo-1-i0zf2  Stopped


INFO:root:copying /tmp/tmpsse2jnk6/model/model/1/saved_model.pb -> /tmp/tmpsse2jnk6/artifacts/model/model/1
INFO:root:copying /tmp/tmpsse2jnk6/compressed_artifacts/output.tar.gz -> /tmp/model
INFO:root:copying /tmp/tmpsse2jnk6/compressed_artifacts/model.tar.gz -> /tmp/model


===== Job Complete =====
